### Step 1: Load packages

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as stats
from statsmodels.stats.power import TTestIndPower
import matplotlib.pyplot as plt
%matplotlib inline
from google.cloud import bigquery
from google.cloud import bigquery_storage
import datetime as dt
from datetime import date, timedelta
import re
from math import ceil
import pingouin as pg
import warnings
warnings.filterwarnings(action = 'ignore') # Suppresses pandas warnings
from IPython.display import display
from tqdm.notebook import tqdm_notebook # Displays progress bars when querying data from BQ
from run_sql_queries import run_query_func

g:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_dag\venv_sb\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.11) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


### Step 2: Define some input parameters to query the relevant test data

In [2]:
# General inputs
query_path = 'G:\My Drive\APAC\Autopricing\Switchback Testing\switchback_test_automation\sql_queries\data_extraction_queries_automated_script.sql' # This is the path to the BQ script that pulls test data
df_raw_data_tbl_name = 'ab_test_individual_orders_cleaned_switchback_tests' # This is the table that contains the cleaned data of switchback tests
alpha_lvl = 0.05 # Set the alpha level below which p-values are statistically significant
p_val_tbl_id = 'dh-logistics-product-ops.pricing.p_vals_switchback_tests' # The table containing the p-values of metrics. This table will be uploaded to BQ at the end of the script
# Define the list of KPIs
col_list = [
    'actual_df_paid_by_customer', 'gfv_local', 'gmv_local', 'commission_local', 'joker_vendor_fee_local', # Customer KPIs (1)
    'sof_local', 'service_fee_local', 'revenue_local', 'delivery_costs_local', 'gross_profit_local', # Customer KPIs (2)
    'dps_mean_delay', 'delivery_distance_m', 'actual_DT' # Logistics KPIs
]

params = [
    {'exp_name': 'TH_20220721_R_F0_O_Switchback_AA_Test_Trang', 'exp_start': dt.date(2022, 7, 21), 'exp_end': dt.date(2022, 8, 17)},
    {'exp_name': 'PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition', 'exp_start': dt.date(2022, 7, 22), 'exp_end': dt.date(2022, 8, 18)},
]

### Step 3.1: Retrieve the switchback test configurations

In [3]:
client = bigquery.Client(project = 'logistics-data-staging-flat') # Instantiate a BQ client and define the project
bq_storage_client = bigquery_storage.BigQueryReadClient() # Instantiate a BQ storage client

# The switchback_test_configs_bq table gets updated every hour via a scheduled query
sb_test_configs = client.query("""SELECT * FROM `dh-logistics-product-ops.pricing.switchback_test_configs_bq`""")\
    .result()\
    .to_dataframe(bqstorage_client = bq_storage_client, progress_bar_type = 'tqdm_notebook')

Downloading:   0%|          | 0/2 [00:00<?, ?rows/s]

### Step 3.2: Extract the scheme IDs from between the curly brackets

In [4]:
# Apply the extraction function on the "scheme_id_on" and "scheme_id_off" columns
sb_test_configs['scheme_id_on'] = sb_test_configs['scheme_id_on'].apply(lambda x: re.findall('\{(.*?)\}', x)[0])
sb_test_configs['scheme_id_off'] = sb_test_configs['scheme_id_off'].apply(lambda x: re.findall('\{(.*?)\}', x)[0])


### Step 3.3: Create a list of dicts storing the config info of the switchback tests

In [5]:
# Declare an empty dict that will be contain the details of a particular switchback test in each for loop iteration
# The keys of the dict are the column names of sb_test_configs
test_config_dict = {}
keys = list(sb_test_configs.columns)

# Declare an empty list that will contain all the dicts storing the test config information
test_config_lod = []

# Populate the list of dicts (lod) with the test configuration info
for i in range(0, len(sb_test_configs.index)): # Enumerate over the number of tests
    for key in keys: # Populate an intermediary dict with the config info of the test belonging to the current iteration
        test_config_dict[key] = sb_test_configs[key][i]
    test_config_lod.append(test_config_dict) # Append the intermediary dict to the list of dicts
    test_config_dict = {} # Empty the dict so that it can be populated again

### Step 3.4: Amend the structure of the list of dicts so that "zone" and "scheme" columns contain lists instead of strings

In [6]:
for test in range(0, len(test_config_lod)): # Iterate over the test dicts
    for key in ['zone_name_vendor_excl', 'zone_name_customer_excl', 'scheme_id_on', 'scheme_id_off']: # Iterate over these keys specifically to change their contents to a list
        if test_config_lod[test][key] == None: # If the value of the key is None, change it to an empty list
            test_config_lod[test][key] = []
        else: 
            test_config_lod[test][key] = test_config_lod[test][key].split(', ') # Split the components of the string into list elements
        
        if 'scheme_id' in key: # If the key being accessed contains the word "scheme_id", change the components of the list to integers using list comprehension
            test_config_lod[test][key] = [int(sch) for sch in test_config_lod[test][key]]
        else:
            pass

## <center>END OF INPUT SECTION. NO NEED TO ADJUST ANY CODE FROM HERE ONWARDS</center> ##

### Step 4: Run the queries that pull the data of Switchback tests

In [5]:
run_query_func(query_path)

g:\My Drive\APAC\Autopricing\Switchback Testing\Switchback_AA_Tests\venv_switchback_aa_tests\lib\site-packages\google\auth\_default.py:81: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. We recommend you rerun `gcloud auth application-default login` and make sure a quota project is added. Or you can use service accounts instead. For more information about service accounts, see https://cloud.google.com/docs/authentication/
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


The SQL script was executed successfully at 2022-08-25 14:58:15.277884 



### Step 5.1: Get the curated data from the resulting table

In [7]:
df_raw_data = client.query("""SELECT * FROM `dh-logistics-product-ops.pricing.{}`"""\
    .format(df_raw_data_tbl_name))\
    .result()\
    .to_dataframe(bqstorage_client = bq_storage_client, progress_bar_type = 'tqdm_notebook')

Downloading:   0%|          | 0/296226 [00:00<?, ?rows/s]

### Step 5.2: Change the data types of columns in the dataset

In [8]:
# Define the start of the data frame where the data types of columns need to be changed 
col_start = np.where(df_raw_data.columns == 'exchange_rate')[0][0]

# Change data types --> df[df.cols = specific cols].apply(pd.to_numeric)
df_raw_data[df_raw_data.columns[col_start:]] = df_raw_data[df_raw_data.columns[col_start:]].apply(pd.to_numeric, errors = 'ignore')

### Step 6: Define a function that changes the font color based on the p-value

In [9]:
# Define a function that changes the font color based on the p-value
def color_sig_green(val):
    """
    Takes a scalar and returns a string with the css property 'color: green' for statistically significant values, red otherwise
    """
    if pd.isna(val) == False:
        if val <= alpha_lvl:
            color = 'lime'
        else:
            color = 'red'
    else:
        color = 'white'
    return 'color: %s' % color

### Step 8:  Analyze if using smaller time intervals produces insignificant deltas between the variation and control

#### Step 8.1: Create a function that extracts the clean data of one test

In [10]:
# Create a function that extracts the clean data of one test
def df_clean_func(df, test_name):
    """
    A function that cleans the data by filtering for the relevant orders based on the test configurations in this G-sheet
    https://docs.google.com/spreadsheets/d/1JeHPeEFUhDaatvCJqqWBgLxRbe_7kLQkObj-lGEX5X0/edit#gid=0
    """
    pos = [iter['test_name'] for iter in test_config_lod].index(test_name) # Get the index of the test's data in test_config_lod
    
    df_temp = df[
        (df['target_group'] != 'Non_TG') &
        (df['test_name'] == test_config_lod[pos]['test_name']) &
        (~ df['zone_name_vendor'].isin(test_config_lod[pos]['zone_name_vendor_excl'])) &
        (~ df['zone_name_customer'].isin(test_config_lod[pos]['zone_name_customer_excl'])) &
        ((df['scheme_id'].isin(test_config_lod[pos]['scheme_id_on'])) | (df['scheme_id'].isin(test_config_lod[pos]['scheme_id_off']))) &
        (df['order_placed_at_local'].dt.date.between(test_config_lod[pos]['test_start'], test_config_lod[pos]['test_end']))
    ] # Filter out Non_TG orders because they will contain irrelevant price schemes

    # We will add a supplementary column to "df_temp" in the for loop below, so we need to create a function with the conditions
    on_off_conditions = [
        (df_temp['scheme_id'].isin(test_config_lod[pos]['scheme_id_on'])),
        (df_temp['scheme_id'].isin(test_config_lod[pos]['scheme_id_off']))
    ]

    # Add a supplementary column to indicate if the order belonged to an 'On' or 'Off' day
    df_temp['on_or_off_day'] = np.select(on_off_conditions, ['On', 'Off'])

    return df_temp

#### Step 8.2: Create a mapping data frame that has the 'On'/'Off' flag for a user-specified time interval

In [11]:
def hr_interval_func_random(sb_interval):
    bins = int(24 / sb_interval) # The number of bins by which we will divide the range from 0 to 24. A 2-hour switchback interval will have 12 bin --> [0, 2), [2, 4), [4, 6), ... [22, 24)
    if sb_interval >= 1:
        end_of_range = 25
    elif sb_interval == 0.5:
        end_of_range = 24.5
    elif sb_interval == 0.25:
        end_of_range = 24.25
    df_mapping = pd.DataFrame(data = {
            'hr_interval': list(pd.cut(np.arange(0, end_of_range, sb_interval), bins = bins, right = False)) # The bins should be closed from the left
        }
    )

    # Drop duplicates
    df_mapping.drop_duplicates(inplace = True)

    unique_intervals = df_mapping['hr_interval'].unique()
    on_off_list = ['On', 'Off'] * ceil(len(unique_intervals) / 2) # Create the full list that the rng.choice would choose from

    rng = np.random.default_rng()
    df_mapping['treatment_status_by_time'] = rng.choice(on_off_list, replace = False, axis = 0, size = len(df_mapping))
    return df_mapping

In [12]:
def hr_interval_date_func_random(test_start, test_length, sb_interval):
    m = []
    date_iter = test_start # Start date of the test in datetime format
    for i in range(0, test_length): # The length of a test in days
        y = hr_interval_func_random(sb_interval) # The switchback window size
        y['sim_run'] = i + 1
        y['order_created_date_local'] = date_iter
        date_iter = date_iter + timedelta(days = 1)
        m.append(y)

    m = pd.concat(m)
    m.reset_index(inplace = True, drop = True)
    return m

In [29]:
def day_interval_date_func_random(test_start, test_length): # np.arange has hard-coded 25 and 24 because this function is intended for randomizing the daily assignment of treatment
    rng = np.random.default_rng()
    on_off_list = ['On', 'Off'] * ceil(test_length / 2) # Create the full list that the rng.choice would choose from
    df_mapping = pd.DataFrame(
        data = {
            'hr_interval': list(pd.cut(np.arange(0, 25, 24), bins = 1, right = False))[0], # The bins should be closed from the left
            'sim_run': np.arange(1, test_length + 1, 1),
            'order_created_date_local': pd.date_range(start = test_start, periods = 28).date.tolist(),
        }
    )
    df_mapping['treatment_status_by_time'] = rng.choice(on_off_list, replace = False, axis = 0, size = test_length)
    return df_mapping

In [14]:
def hr_interval_func_deterministic(sb_interval):
    bins = int(24 / sb_interval) # The number of bins by which we will divide the range from 0 to 24. A 2-hour switchback interval will have 12 bin --> [0, 2), [2, 4), [4, 6), ... [22, 24)
    if sb_interval >= 1:
        end_of_range = 25
    elif sb_interval == 0.5:
        end_of_range = 24.5
    elif sb_interval == 0.25:
        end_of_range = 24.25
    df_mapping = pd.DataFrame(data = {
            'hr_interval': list(pd.cut(np.arange(0, end_of_range, sb_interval), bins = bins, right = False)) # The bins should be closed from the left
        }
    )

    unique_intervals = df_mapping['hr_interval'].unique()
    tp = list(zip(unique_intervals, ['On', 'Off'] * ceil(len(unique_intervals) / 2)))

    df_mapping['treatment_status_by_time'] = df_mapping['hr_interval'].map(dict(tp))
    return df_mapping.drop_duplicates()

# Create a function that returns the right hr_interval from df_mapping for any given number
def check_right_interval(num, col):
    for i in col:
        if num in i:
            return i
        else:
            pass

#### Step 8.3: Create a function that computes the per order metrics, total metrics, and p-values based on the newly defined 'On'/'Off' granularity

#### MWU test

In [67]:
def df_calc_func(data, test_name, test_start, test_end, sb_interval_hr, sb_groupby_col, hr_or_day):
    """
    This function computes the per order metrics, total metrics, and p-values based on a newly defined 'On'/'Off' flag
    """
    # Get the data frame on which you will operate
    df_play = df_clean_func(df = data, test_name = test_name)
    
    # Create a new column with the located created_date of the order 
    df_play['order_created_date_local'] = df_play['order_placed_at_local'].dt.date

    # Filter for the data between test_start and test_end
    df_play = df_play[df_play['order_created_date_local'].between(test_start, test_end)]

    # Create the mapping data frame that has the 'On'/'Off' flag for each time interval
    df_mapping = hr_interval_date_func_random(test_start = test_start, test_length = (test_end - test_start).days + 1, sb_interval = sb_interval_hr) if hr_or_day == 'hr'\
        else day_interval_date_func_random(test_start = test_start, test_length = (test_end - test_start).days + 1)

    # Get the hour from the "order_placed_at_local" column
    df_play['hr_of_day'] = round(df_play['order_placed_at_local'].dt.hour + df_play['order_placed_at_local'].dt.minute / 60 + df_play['order_placed_at_local'].dt.second / 3600, 2)
    
    # Get the right interval using the "check_right_interval" function
    df_play['hr_interval'] = df_play.apply(lambda x: check_right_interval(x['hr_of_day'], df_mapping['hr_interval'].unique()), axis = 1)
    
    # Map the "hr_interval" to the correct treatment_status
    df_play = pd.merge(left = df_play, right = df_mapping, how = 'left', on = ['hr_interval', 'order_created_date_local'])

    # If the test_name == 'PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition', convert the 'On' values of the Canlubang zone to 'Off' and vice-versa
    def treatment_status_switching_func(zone_name, treatment_status_by_time):
        if zone_name == 'Cabuyao':
            pass
        elif zone_name == 'Canlubang' and treatment_status_by_time == 'On':
            treatment_status_by_time = 'Off'
        elif zone_name == 'Canlubang' and treatment_status_by_time == 'Off':
            treatment_status_by_time = 'On'
        return treatment_status_by_time

    if test_name == 'PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition':
        df_play['treatment_status_by_time'] = df_play\
            .apply(lambda x: treatment_status_switching_func(x['zone_name_customer'], x['treatment_status_by_time']), axis = 1)

    # Calculate the "total" metrics and rename the column label to "df_per_order_metrics"
    df_play_tot = round(df_play.groupby(['test_name', 'on_or_off_day'])[col_list[:-3]].sum(), 2) if sb_groupby_col == 'on_or_off_day' else\
        round(df_play.groupby(['test_name', 'treatment_status_by_time'])[col_list[:-3]].sum(), 2)
    df_play_tot['order_count'] = df_play.groupby(['test_name', 'on_or_off_day'])['order_id'].nunique() if sb_groupby_col == 'on_or_off_day' else\
        df_play.groupby(['test_name', 'treatment_status_by_time'])['order_id'].nunique()
    df_play_tot = df_play_tot.rename_axis(['df_tot_metrics'], axis = 1)
    
    # Calculate the "total" metrics and rename the column label to "df_per_order_metrics"
    df_play_per_order_cust_kpis = df_play_tot.copy()

    for iter_col in df_play_per_order_cust_kpis.columns[:-1]:
        df_play_per_order_cust_kpis[iter_col] = round(df_play_per_order_cust_kpis[iter_col] / df_play_per_order_cust_kpis['order_count'], 4)

    df_play_per_order_log_kpis = round(df_play.groupby(['test_name', 'on_or_off_day'])[col_list[-3:]].mean(), 2) if sb_groupby_col == 'on_or_off_day' else\
        round(df_play.groupby(['test_name', 'treatment_status_by_time'])[col_list[-3:]].mean(), 2) 
    df_play_per_order = pd.concat([df_play_per_order_cust_kpis, df_play_per_order_log_kpis], axis = 1)
    df_play_per_order = df_play_per_order.rename_axis(['df_per_order_metrics'], axis = 1)
    
    # Add a thousands separator to df_play_tot
    for i in df_play_tot.columns:
        df_play_tot[i] = df_play_tot[i].map('{:,}'.format)

    # Now, we need to calculate the p-values. Create two sub-data frames for the 'On' and 'Off' days
    df_play_on_orders = df_play[df_play['on_or_off_day'] == 'On'] if sb_groupby_col == 'on_or_off_day' else df_play[df_play['treatment_status_by_time'] == 'On']
    df_play_off_orders = df_play[df_play['on_or_off_day'] == 'Off'] if sb_groupby_col == 'on_or_off_day' else df_play[df_play['treatment_status_by_time'] == 'Off']

    df_play_final_pval = []
    pval_dict = {} # Initialize an empty dict that will contain the p-value of each KPI
    for i in col_list:
        pval = round(stats.mannwhitneyu(x = df_play_on_orders[i], y = df_play_off_orders[i], alternative = 'two-sided', nan_policy = 'omit')[1], 4)
        pval_dict[i] = pval

    df_play_final_pval.append(pval_dict)

    # Change the list of dicts "df_final_pval" to a data frame
    df_play_final_pval = pd.DataFrame(df_play_final_pval)\
        .assign(test_name = test_name)\
        .set_index('test_name')
    
    return df_mapping, df_play, df_play_on_orders, df_play_off_orders, df_play_per_order, df_play_tot, df_play_final_pval

#### Step 8.4: Execute the "df_calc_func" function with different sb_intervals

In [111]:
df_mapping, df_play, df_play_on_orders, df_play_off_orders, df_play_per_order, df_play_tot, df_play_final_pval = df_calc_func(
            data = df_raw_data, 
            test_name = 'TH_20220721_R_F0_O_Switchback_AA_Test_Trang', 
            test_start = dt.date(2022, 7, 21), 
            test_end = dt.date(2022, 8, 17),
            sb_interval_hr = 24,
            sb_groupby_col = 'treatment_status_by_time',
            hr_or_day = 'day'
        )

In [112]:
df_play_final_pval.style.format(precision=4).applymap(color_sig_green)

,actual_df_paid_by_customer,gfv_local,gmv_local,commission_local,joker_vendor_fee_local,sof_local,service_fee_local,revenue_local,delivery_costs_local,gross_profit_local,dps_mean_delay,delivery_distance_m,actual_DT
test_name,,,,,,,,,,,,,
TH_20220721_R_F0_O_Switchback_AA_Test_Trang,0.0296,0.5601,0.0078,0.2405,0.5621,0.6935,1.0000,0.1061,0.0000,0.0000,0.0000,0.7809,0.0000


In [114]:
num_sims = 30

df_pval_mwu_test_sims = []
for i in range(0, num_sims): # 30 simulations
    df_pval_mwu_test_iter = []

    for test in params:
        df_mapping, df_play, df_play_on_orders, df_play_off_orders, df_play_per_order, df_play_tot, df_play_final_pval = df_calc_func(
            data = df_raw_data, 
            test_name = test['exp_name'], 
            test_start = test['exp_start'], 
            test_end = test['exp_end'],
            sb_interval_hr = 24,
            sb_groupby_col = 'treatment_status_by_time', 
            hr_or_day = 'day'
        )
        df_pval_mwu_test_iter.append(df_play_final_pval)
    df_pval_mwu_test_iter = pd.concat(df_pval_mwu_test_iter)
    df_pval_mwu_test_iter['sim_run'] = i + 1
    df_pval_mwu_test_iter.set_index('sim_run', append = True, inplace = True)

    df_pval_mwu_test_sims.append(df_pval_mwu_test_iter)

df_pval_mwu_test_sims = pd.concat(df_pval_mwu_test_sims)
df_pval_mwu_test_sims.head(5).style.format(precision = 4).applymap(color_sig_green)
        

,,actual_df_paid_by_customer,gfv_local,gmv_local,commission_local,joker_vendor_fee_local,sof_local,service_fee_local,revenue_local,delivery_costs_local,gross_profit_local,dps_mean_delay,delivery_distance_m,actual_DT
test_name,sim_run,,,,,,,,,,,,,
TH_20220721_R_F0_O_Switchback_AA_Test_Trang,1,0.6934,0.0047,0.0401,0.0271,0.6842,0.8476,1.0000,0.0173,0.3223,0.3003,0.0000,0.0000,0.0000
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition,1,0.2743,0.8489,0.5966,0.7480,0.9658,0.0329,1.0000,0.3675,0.9183,0.5944,0.0000,0.8792,0.2187
TH_20220721_R_F0_O_Switchback_AA_Test_Trang,2,0.0001,0.2228,0.6883,0.3770,0.1863,0.6667,1.0000,0.0076,0.0000,0.0000,0.0000,0.6472,0.0000
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition,2,0.6495,0.4509,0.2592,0.3976,0.7036,0.7188,1.0000,0.4314,0.1341,0.1965,0.0418,0.9751,0.2365
TH_20220721_R_F0_O_Switchback_AA_Test_Trang,3,0.0014,0.0016,0.0115,0.0070,0.4185,0.2747,1.0000,0.0002,0.0000,0.0000,0.0000,0.0168,0.0000


In [115]:
# Calculate the average p-values from the simulations
df_pval_mwu_test_sims.groupby('test_name').mean().style.format(precision = 4).applymap(color_sig_green)

,actual_df_paid_by_customer,gfv_local,gmv_local,commission_local,joker_vendor_fee_local,sof_local,service_fee_local,revenue_local,delivery_costs_local,gross_profit_local,dps_mean_delay,delivery_distance_m,actual_DT
test_name,,,,,,,,,,,,,
PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition,0.3201,0.4449,0.4455,0.4871,0.5159,0.4235,1.0000,0.4272,0.1857,0.2669,0.0241,0.5358,0.3062
TH_20220721_R_F0_O_Switchback_AA_Test_Trang,0.2149,0.2177,0.2118,0.2086,0.4789,0.5462,1.0000,0.1817,0.0550,0.0553,0.0033,0.1398,0.0387


In [123]:
def num_sims_sig_func(test_name, df):
    df_temp = df[(df.index.get_level_values('test_name') == test_name)].apply(lambda x: x[x <= 0.05].count()).to_frame(name = 'num_sims_sig_' + test_name[0:2])
    df_temp['pct_times_sig'] = round((df_temp.iloc[:, 0] / num_sims) * 100, 2).astype(str) + '%'
    
    return df_temp

display(num_sims_sig_func(test_name='TH_20220721_R_F0_O_Switchback_AA_Test_Trang', df=df_pval_mwu_test_sims))
display(num_sims_sig_func(test_name='PH_20220721_R_F0_O_SB_AA_Test_Calamba_No_Condition', df=df_pval_mwu_test_sims))

,num_sims_sig_TH,pct_times_sig
actual_df_paid_by_customer,17,56.67%
gfv_local,15,50.0%
gmv_local,14,46.67%
commission_local,16,53.33%
joker_vendor_fee_local,2,6.67%
sof_local,2,6.67%
service_fee_local,0,0.0%
revenue_local,17,56.67%
delivery_costs_local,25,83.33%
gross_profit_local,25,83.33%


,num_sims_sig_PH,pct_times_sig
actual_df_paid_by_customer,12,40.0%
gfv_local,3,10.0%
gmv_local,3,10.0%
commission_local,3,10.0%
joker_vendor_fee_local,0,0.0%
sof_local,1,3.33%
service_fee_local,0,0.0%
revenue_local,2,6.67%
delivery_costs_local,15,50.0%
gross_profit_local,12,40.0%
